# FullControl design template

*<<< check out demo models [here](https://github.com/FullControlXYZ/fullcontrol/tree/master/models/README.md) >>>*
  
run all cells in this notebook, or press shift+enter to run each cell sequentially 

if you change one of the code cells, make sure you run it and all subsequent cells again (in order)

*this document is a jupyter notebook - if they're new to you, check out how they work: [link](https://www.google.com/search?q=ipynb+tutorial), [link](https://jupyter.org/try-jupyter/retro/notebooks/?path=notebooks/Intro.ipynb), [link](https://colab.research.google.com/)*

In [30]:
import fullcontrol as fc
import math

In [31]:
def circle_infill(diameter:float,start_point:fc.Point,centre_point:fc.Point,line_space:float):
    import math
    radius_c=diameter/2
    steps=[]
    steps.append(start_point)
    for space in range(line_space,diameter,line_space):
        if radius_c-space>0:
            steps.append(fc.move_polar(geometry=start_point,centre=centre_point,radius=radius_c-space,angle=0))
            steps.append(fc.move_polar(geometry=start_point,centre=centre_point, radius=radius_c-space,angle=math.acos((radius_c-space)/radius_c)))            
            steps.append(fc.move_polar(geometry=start_point,centre=centre_point, radius=radius_c-(2*space),angle=math.acos((radius_c-(2*space))/radius_c)))
            steps.append(fc.move_polar(geometry=start_point,centre=centre_point, radius=radius_c-(2*space),angle=0))
        else:
            steps.append(fc.move_polar(geometry=start_point,centre=centre_point,radius=radius_c-space,angle=0))
            steps.append(fc.move_polar(geometry=start_point,centre=centre_point, radius=radius_c-space,angle=math.acos((radius_c-space)/radius_c)))
            print(radius_c,space)            
            steps.append(fc.move_polar(geometry=start_point,centre=centre_point, radius=radius_c-(2*space),angle=math.acos((space)/radius_c)))
            steps.append(fc.move_polar(geometry=start_point,centre=centre_point, radius=radius_c-(2*space),angle=0))
    return steps

In [32]:
# printer/gcode parameters

design_name = 'square'
nozzle_temp = 0
bed_temp = 0
print_speed = 300
fan_percent = 0
printer_name='stoneflower3d' # generic / ultimaker2plus / prusa_i3 / ender_3 / cr_10 / bambulab_x1 / toolchanger_T0

In [33]:
# design parameters

EW = 0.5 # extrusion width
EH = 0.25 # extrusion height (and layer height)
initial_z = EH*1 # initial nozzle position is set to 1x the extrusion height to get a bit of 'squish' for good bed adhesion
layers = 1
dia=20 #diameter of the cylinder


In [34]:
# generate the design (make sure you've run the above cells before running this cell)
initial_xyz=fc.Point(x=200, y=200, z=initial_z)
steps = []
#Generate the layer patterns such that no travel is require between the layers
for layer in range(layers):
        layer1=[]
        first_point=fc.move(initial_xyz,fc.Vector(y=dia/2,z=EH*layer))
        centre_point = fc.move(initial_xyz,fc.Vector(x=dia/2,y=dia/2,z=EH*layer))
        radius = dia/2
        start_angle =math.pi
        segments = 32
        clockwise = False
        circle_steps = fc.circleXY(centre_point, radius, start_angle, segments, clockwise)
        line_spacing = 1        
        infill_steps = circle_infill(dia,first_point,centre_point,line_spacing)
        layer1.append(circle_steps)
        layer1.append(infill_steps)
        print(centre_point)
        print(circle_steps[0])        
    
        layer2=[]
        first_point=fc.move(initial_xyz,fc.Vector(y=dia,z=EH*layer))
        rectangle_steps = fc.rectangleXY(first_point, dia, -dia, cw = True)
        start_point = fc.move(first_point,fc.Vector(x=EW,y=-EW))
        direction = fc.Vector(x=1,y=0)
        amplitude = -(dia-2*EW)
        line_spacing = 1
        periods = dia//2
        extra_half_period = False
        extra_end_line = False
        infill_steps = fc.squarewaveXY(start_point, direction, amplitude, line_spacing, periods, extra_half_period, extra_end_line)
        layer2.append(rectangle_steps)
        layer2.append(infill_steps)
        
        layer3=[]
        first_point=fc.move(initial_xyz,fc.Vector(x=dia,z=EH*layer))
        rectangle_steps = fc.rectangleXY(first_point, -dia, dia, cw = True)
        start_point = fc.move(first_point,fc.Vector(x=-EW,y=EW))
        direction = fc.Vector(x=0,y=1)
        amplitude = (dia-2*EW)
        line_spacing = 1
        periods = dia//2
        extra_half_period = False
        extra_end_line = False
        infill_steps = fc.squarewaveXY(start_point, direction, amplitude, line_spacing, periods, extra_half_period, extra_end_line)
        layer3.append(rectangle_steps)
        layer3.append(infill_steps)

        layer4=[]
        first_point=fc.move(initial_xyz,fc.Vector(x=dia,y=dia,z=EH*layer))
        rectangle_steps = fc.rectangleXY(first_point, -dia, -dia, cw = True)
        start_point = fc.move(first_point,fc.Vector(x=-EW,y=-EW))
        direction = fc.Vector(x=-1,y=0)
        amplitude = (dia-2*EW)
        line_spacing = 1
        periods = dia//2
        extra_half_period = False
        extra_end_line = False
        infill_steps = fc.squarewaveXY(start_point, direction, amplitude, line_spacing, periods, extra_half_period, extra_end_line)
        layer4.append(rectangle_steps)
        layer4.append(infill_steps)
        layertypes=[layer1,layer2,layer3,layer4]
        steps.extend(layertypes[layer%len(layertypes)]) 

#steps = fc.move(rectangle_steps, fc.Vector(z=EH), copy=True, copy_quantity=layers)

10.0 10
10.0 11


ValueError: math domain error

In [ ]:
# preview the design

fc.transform(steps, 'plot', fc.PlotControls(style='line', zoom=0.7))
# hover the cursor over the lines in the plot to check xyz positions of the points in the design

# uncomment the next line to create a plot with real heights/widths for extruded lines to preview the real 3D printed geometry
fc.transform(steps, 'plot', fc.PlotControls(style='tube', zoom=0.7, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))

# uncomment the next line to create a neat preview (click the top-left button in the plot for a .png file) - post and tag @FullControlXYZ :)
# fc.transform(steps, 'plot', fc.PlotControls(neat_for_publishing=True, zoom=0.5, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))


In [ ]:
# generate and save gcode

gcode_controls = fc.GcodeControls(
    printer_name=printer_name,
    save_as=design_name,
    initialization_data={
        'primer': 'front_lines_then_y',
        'print_speed': print_speed,
        'nozzle_temp': nozzle_temp,
        'bed_temp': bed_temp,
        'fan_percent': fan_percent,
        'extrusion_width': EW,
        'extrusion_height': EH})
gcode = fc.transform(steps, 'gcode', gcode_controls)

#### please tell us what you're doing with FullControl!

- tag FullControlXYZ on social media ([twitter](https://twitter.com/FullControlXYZ), [instagram](https://www.instagram.com/fullcontrolxyz/), [linkedin](https://www.linkedin.com/in/andrew-gleadall-068587119/), [tiktok](https://www.tiktok.com/@fullcontrolxyz))
- email [info@fullcontrol.xyz](mailto:info@fullcontrol.xyz)
- post on the [subreddit](https://reddit.com/r/fullcontrol)
- post in the [github discussions or issues tabs](https://github.com/FullControlXYZ/fullcontrol/issues)

in publications, please cite the original FullControl paper and the github repo for the new python version:

- Gleadall, A. (2021). FullControl GCode Designer: open-source software for unconstrained design in additive manufacturing. Additive Manufacturing, 46, 102109. 
- Gleadall, A. and Leas, D. (2023). FullControl [electronic resource: python source code]. available at: https://github.com/FullControlXYZ/fullcontrol